In [1]:
import cv2
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('../'))
import pandas as pd
import datetime
import random
from utilities.helper_functions import helper_functions as hf
import config

In [2]:
video_path = config.remote_video_path
processed_data_path = config.successful_trial_path
reference_path = [config.marker_path + dir for dir in os.listdir(config.marker_path)]


def show_successfull_trials(video, df, offset):
     # iterate through df and for each successfull trial get the start time in seconds
    reference_image = cv2.imread(reference_path[0])
    transformation, num_match = hf.find_matching_transformation(video.read()[1], reference_path)
    for index, row in df.iterrows():
        if row['successful_trials']:
            transformation_new, num_match_new = hf.find_matching_transformation(video.read()[1], reference_path)
            if num_match_new > num_match or num_match_new > 30:
                transformation = transformation_new
                num_match = num_match_new
            start_time_str = row['time_since_start'].split(' ')[2]
            start_time = start_time_str.split(':')
            start_time = float(start_time[0])*3600 + float(start_time[1])*60 + float(start_time[2]) + offset
            end_time = start_time + 15
            # set the video to the start time
            fps = video.get(cv2.CAP_PROP_FPS)
            current_time = float(video.get(cv2.CAP_PROP_POS_FRAMES)) / fps
            time_offset = start_time - current_time
            for i in range(int(time_offset * video.get(cv2.CAP_PROP_FPS))):
                video.read()

            # find the frames where mouse R and L break the beam
            R_time_offset = row['R']
            L_time_offset = row['L']
            
            # find the mice names
            name1 = row['names'][:2]
            name2 = row['names'][2:]
            # display the video until the end time
            
            while True:
                success, image = video.read()
                if success == False or float(video.get(cv2.CAP_PROP_POS_FRAMES))/fps > end_time:
                    break
                # add a countdown to the video 
                time_since_start_seconds = float(video.get(cv2.CAP_PROP_POS_FRAMES))/fps
                time_since_start = str(datetime.timedelta(seconds=time_since_start_seconds))
                transformed = cv2.warpPerspective(image, transformation, (reference_image.shape[1], reference_image.shape[0]))

                cv2.putText(image, start_time_str + ' ' + time_since_start, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                # display the mice names
                cv2.putText(image, 'trial: ' + str(index), (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, name1 + ' ' + name2, (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, str(df['cue_times'])[:18], (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.imshow('frame', image)
                cv2.imshow('transformed', transformed)

                if cv2.waitKey(12) & 0xFF == ord('q'):
                    break

def write_successful_trials(video, df, offset, path):
    # iterate through df and for each successfull trial get the start time in seconds
    reference_image = cv2.imread(reference_path[0])
    transformation, num_match = hf.find_matching_transformation(video.read()[1], reference_path)
    # create video writer
    fps = video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    for index, row in df.iterrows():
        if row['successful_trials']:
            transformation_new, num_match_new = hf.find_matching_transformation(video.read()[1], reference_path)
            if num_match_new > num_match or num_match_new > 30:
                transformation = transformation_new
                num_match = num_match_new
            start_time_str = row['time_since_start'].split(' ')[2]
            start_time = start_time_str.split(':')
            start_time = float(start_time[0])*3600 + float(start_time[1])*60 + float(start_time[2]) + offset
            end_time = start_time + 15
            # set the video to the start time
            fps = video.get(cv2.CAP_PROP_FPS)
            current_time = float(video.get(cv2.CAP_PROP_POS_FRAMES)) / fps
            time_offset = start_time - current_time
            for i in range(int(time_offset * video.get(cv2.CAP_PROP_FPS))):
                video.read()

            # find the mice names
            name1 = row['names'][:2]
            name2 = row['names'][2:]
            # write the video to a new file until the end time
            # check if folder exists, if not create it
            if not os.path.exists(path):
                os.makedirs(path)
            out = cv2.VideoWriter(path + '_trial_' +str(index) + '.mp4', fourcc, fps, (reference_image.shape[1], reference_image.shape[0]))

            while True:
                success, image = video.read()
                if success == False or float(video.get(cv2.CAP_PROP_POS_FRAMES))/fps > end_time:
                    break
                # add a countdown to the video 
                transformed = cv2.warpPerspective(image, transformation, (reference_image.shape[1], reference_image.shape[0]))
                out.write(transformed)
                if cv2.waitKey(12) & 0xFF == ord('q'):
                    break
            out.release()


In [3]:
# get list of video names from the folder of processed CSVs
csv_times = [ hf.process_time_string(name.split('_')[0]) for name in os.listdir(processed_data_path)]
csv_paths = [name for name in os.listdir(processed_data_path)]
# iterate through each csv and find the closest video name
video_times = [hf.process_time_string(name.split('_')[0]) for name in os.listdir(video_path)]
video_paths = [name for name in os.listdir(video_path)]
csv_vid_match = dict()
for csv_time in csv_times:
    closest_time = video_times[0]
    closest_path = video_paths[0]
    for i, video_time in enumerate(video_times):
        if abs((csv_time - video_time).total_seconds()) <  abs((csv_time - closest_time).total_seconds()):
            closest_time = video_time
            closest_path = video_paths[i]
    total_delta = (csv_time - closest_time).total_seconds()
    if total_delta < 10:
        csv_vid_match[csv_paths[csv_times.index(csv_time)]] = {
            'video_name': closest_path,
            'delta_seconds': total_delta
        }


significant_trials = []
for csv_name in csv_vid_match.keys():
    df = pd.read_csv(processed_data_path + '/' + csv_name)
    if len(df['successful_trials']) >= 10:
        significant_trials.append(csv_name)
significant_trials.sort()
significant_trials.reverse()
# randoize order
random.shuffle(significant_trials)


    

In [4]:
# iterate throught each video and load the corresponding video and csv and then write the video with the successful trials
for csv_name in significant_trials:
    # check if folder for trial exists, if so skip
    if os.path.exists(config.video_output_path + csv_name.split('_')[0] + '/'):
        continue
    video = cv2.VideoCapture(video_path + csv_vid_match[csv_name]['video_name'])
    df = pd.read_csv(processed_data_path + '/' + csv_name)
    if len(df['successful_trials']) < 10:
        continue
    write_successful_trials(video, df, csv_vid_match[csv_name]['delta_seconds'], config.video_output_path + csv_name.split('_')[0] + '/')
